In [1]:
import json
import os

# import the data from File
filepath = os.path.abspath(os.path.join(os.getcwd(), '..', 'example', 'json', 'complete_drawflow_refract.json'))
print(filepath)

with open(filepath, 'r+') as f:
    json_data = json.load(f)

/Users/pyrokar/Documents/Coding/ensys-gui-new/example/json/complete_drawflow_refract.json


In [5]:
from backend.app.ensys.components import EnFlow, EnInvestment, EnConverter, EnSink, EnSource, EnGenericStorage, EnBus, \
    EnEnergysystem, EnModel


def create_io_data(json_component) -> (dict, dict):
    # build component_data["inputs"]
    component_data = json_component["data"]

    input_data = {}
    for input_name in json_component["inputs"]:
        target_bus_id = json_component["inputs"][input_name]['connections'][0]["node"]
        target_bus_name = json_data[target_bus_id]["name"]

        flow_data = component_data["connections"]["inputs"][input_name]["formInfo"]

        if flow_data["investment"] is True:
            flow_data["nominal_value"] = EnInvestment(
                maximum=flow_data["maximum"],
                minimum=flow_data["minimum"],
                ep_costs=flow_data["ep_costs"],
                existing=flow_data["existing"],
                nonconvex=flow_data["nonconvex"],
                offset=flow_data["offset"],
                overall_maximum=flow_data["overall_maximum"],
                overalL_minimum=flow_data["overall_minimum"],
                lifetime=flow_data["lifetime"],
                age=flow_data["age"],
                interest_rate=flow_data["interest_rate"],
                fixed_costs=flow_data["fixed_costs"],
                # custom_attributes="to be done"
            )

        input_data[target_bus_name] = EnFlow(**flow_data)

    # build component_data["outputs"]
    output_data = {}
    for output_name in json_component["outputs"]:
        target_bus_id = json_component["outputs"][output_name]['connections'][0]["node"]
        target_bus_name = json_data[target_bus_id]["name"]

        flow_data = component_data["connections"]["outputs"][output_name]["formInfo"]

        if flow_data["investment"] is True:
            flow_data["nominal_value"] = EnInvestment(
                maximum=flow_data["maximum"],
                minimum=flow_data["minimum"],
                ep_costs=flow_data["ep_costs"],
                existing=flow_data["existing"],
                nonconvex=flow_data["nonconvex"],
                offset=flow_data["offset"],
                overall_maximum=flow_data["overall_maximum"],
                overalL_minimum=flow_data["overall_minimum"],
                lifetime=flow_data["lifetime"],
                age=flow_data["age"],
                interest_rate=flow_data["interest_rate"],
                fixed_costs=flow_data["fixed_costs"],
                # custom_attributes="to be done"
            )

        output_data[target_bus_name] = EnFlow(**flow_data)

    return input_data, output_data


def build_conversion_factors(json_component) -> dict:
    component_ports = json_component["data"]["ports"]
    conversion_factors = {}

    # build conversion_factors
    for input in component_ports["inputs"]:
        input_name = component_ports["inputs"][input["id"]]["code"]

        target_bus_id = json_component["inputs"][input_name]['connections'][0]["node"]
        target_bus_name = json_data[target_bus_id]["name"]

        conversion_value = component_ports["inputs"][input["id"]]["number"]
        if conversion_value is not None:
            conversion_factors[target_bus_name] = conversion_value

    for output in component_ports["outputs"]:
        output_name = component_ports["outputs"][output["id"]]["code"]

        target_bus_id = json_component["outputs"][output_name]['connections'][0]["node"]
        target_bus_name = json_data[target_bus_id]["name"]

        conversion_value = component_ports["outputs"][output["id"]]["number"]
        if conversion_value is not None:
            conversion_factors[target_bus_name] = conversion_value

    return conversion_factors

def convert_gui_json_to_ensys(json_data: dict) -> (dict, dict):
    ensys_es = EnEnergysystem()

    for json_index in json_data:
        print(f"Component: {json_data[json_index]['name']}")
        json_component = json_data[json_index]

        component_class = json_component["class"]
        component_name = json_component["name"]
        component_data = json_component["data"]

        if component_class != "bus":
            input_data, output_data = create_io_data(json_component)
            component_data["inputs"] = input_data
            component_data["outputs"] = output_data

        if component_class in ["converter", "transformer"]:
            conversion_factors = build_conversion_factors(json_component)
            component_data["conversion_factors"] = conversion_factors

            ensys_component = EnConverter(**component_data)

        elif component_class == "sink":
            ensys_component = EnSink(**component_data)

        elif component_class == "source":
            ensys_component = EnSource(**component_data)

        elif component_class == "genericStorage":
            ensys_component = EnGenericStorage(**component_data)

        elif component_class == "bus":
            ensys_component = EnBus(label=component_name)

        else:
            print(f"Component {component_name} is not supported yet")

        ensys_es.add(ensys_component)

    return ensys_es

es_new = convert_gui_json_to_ensys(json_data)

filepath = os.path.abspath(os.path.join(os.getcwd(), '..', 'example', 'json', 'converted_es.json'))
print(filepath)
with open(filepath, 'w+') as f:
    json.dump(es_new.model_dump(), f, indent=4)


Component: SourceA
Component: Transformer
Component: BusA
Component: BusB
Component: BusC
Component: BusD
Component: SinkA
Component: SinkB
Component: SourceB
/Users/pyrokar/Documents/Coding/ensys-gui-new/example/json/converted_es.json
